In [40]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearClassifier(BaseEstimator):
      """
      General class for binary linear classifiers. Implements the predict
      function, which is the same for all binary linear classifiers. There are
      also two utility functions.
      """

      def decision_function(self, X):
            """
            Computes the decision function for the inputs X. The inputs are assumed to be
            stored in a matrix, where each row contains the features for one
            instance.
            """
            return X.dot(self.w)

      def predict(self, X):
            """
            Predicts the outputs for the inputs X. The inputs are assumed to be
            stored in a matrix, where each row contains the features for one
            instance.
            """

            # First compute the output scores
            scores = self.decision_function(X)

            # Select the positive or negative class label, depending on whether
            # the score was positive or negative.
            out = np.select([scores >= 0.0, scores < 0.0],
                            [self.positive_class,
                             self.negative_class])
            return out

      def find_classes(self, Y):
            """
            Finds the set of output classes in the output part Y of the training set.
            If there are exactly two classes, one of them is associated to positive
            classifier scores, the other one to negative scores. If the number of
            classes is not 2, an error is raised.
            """
            classes = sorted(set(Y))
            if len(classes) != 2:
                  raise Exception("this does not seem to be a 2-class problem")
            self.positive_class = classes[1]
            self.negative_class = classes[0]

      def encode_outputs(self, Y):
            """
            A helper function that converts all outputs to +1 or -1.
            """
            return np.array([1 if y == self.positive_class else -1 for y in Y])


class Perceptron(LinearClassifier):
      """
      A straightforward implementation of the perceptron learning algorithm.
      """

      def __init__(self, n_iter=20):
            """
            The constructor can optionally take a parameter n_iter specifying how
            many times we want to iterate through the training set.
            """
            self.n_iter = n_iter

      def fit(self, X, Y):
            """
            Train a linear classifier using the perceptron learning algorithm.
            """

            # First determine which output class will be associated with positive
            # and negative scores, respectively.
            self.find_classes(Y)

            # Convert all outputs to +1 (for the positive class) or -1 (negative).
            Ye = self.encode_outputs(Y)

            # If necessary, convert the sparse matrix returned by a vectorizer
            # into a normal NumPy matrix.
            if not isinstance(X, np.ndarray):
                  X = X.toarray()

            # Initialize the weight vector to all zeros.
            n_features = X.shape[1]
            self.w = np.zeros(n_features)

            # Perceptron algorithm:
            for i in range(self.n_iter):
                  for x, y in zip(X, Ye):

                        # Compute the output score for this instance.
                        score = x.dot(self.w)

                        # If there was an error, update the weights.
                        if y * score <= 0:
                              self.w += y * x

In [41]:
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels.
def read_data(corpus_file):
      X = []
      Y = []
      with open(corpus_file, encoding='utf-8') as f:
            for line in f:
                  _, y, _, x = line.split(maxsplit=3)
                  X.append(x.strip())
                  Y.append(y)
      return X, Y


# Read all the documents.
X, Y = read_data('data/all_sentiment_shuffled.txt')
# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
      TfidfVectorizer(),
      SelectKBest(k=1000),
      Normalizer(),
      # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
      Perceptron()
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.00 sec.
Accuracy: 0.7919.


In [42]:
class PegasosSVC(LinearClassifier):
      """
      Pegasos Support Vector Classifier
      """

      def __init__(self, n_iter=20, lam=1):
            """
            The constructor can optionally take a parameter n_iter specifying how
            many times we want to iterate through the training set.
            """
            self.n_iter = n_iter
            self.lam = lam

      def fit(self, X, Y):
            """
            Train a linear classifier using the perceptron learning algorithm.
            """

            self.find_classes(Y)

            Ye = self.encode_outputs(Y)

            if not isinstance(X, np.ndarray):
                  X = X.toarray()

            # Initialize the weight vector to all zeros.
            n_features = X.shape[1]
            m = X.shape[0]
            self.w = np.zeros(n_features)

            for i in range(1, self.n_iter + 1):

                  j = np.random.choice(m, 1)[0]
                  x, y = X[j], Ye[j]
                  nu = 1. / (self.lam * i)
                  score = self.w.dot(x)
                  if y * score < 1:
                        self.w = (1 - nu * self.lam) * self.w + (nu * y) * x
                  else:
                        self.w = (1 - nu * self.lam) * self.w


In [43]:
pipeline = make_pipeline(
      TfidfVectorizer(),
      SelectKBest(k=1000),
      Normalizer(),
      PegasosSVC(lam=1/len(Xtrain), n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 0.78 sec.
Accuracy: 0.8095.


In [44]:
class PegasosLR(LinearClassifier):
      """
      Pegasos Logistic Regression
      """

      def __init__(self, n_iter=20, lam=1):
            """
            The constructor can optionally take a parameter n_iter specifying how
            many times we want to iterate through the training set.
            """
            self.n_iter = n_iter
            self.lam = lam

      def fit(self, X, Y):
            """
            Train a linear classifier using the perceptron learning algorithm.
            """

            self.find_classes(Y)

            Ye = self.encode_outputs(Y)

            if not isinstance(X, np.ndarray):
                  X = X.toarray()

            # Initialize the weight vector to all zeros.
            n_features = X.shape[1]
            m = X.shape[0]
            self.w = np.zeros(n_features)

            for i in range(1, self.n_iter + 1):
                  j = np.random.choice(m, 1)[0]
                  x, y = X[j], Ye[j]
                  nu = 1. / (self.lam * i)
                  score = self.w.dot(x)
                  loss_gradient = -(y / (1 + np.exp(y * score))) * x
                  self.w = (1 - nu * self.lam) * self.w - nu * loss_gradient


In [45]:
pipeline = make_pipeline(
      TfidfVectorizer(),
      SelectKBest(k=1000),
      Normalizer(),
      PegasosLR(lam=1/len(Xtrain), n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 0.80 sec.
Accuracy: 0.8208.


In [46]:
from scipy.linalg.blas import ddot, dscal, daxpy

class PegasosSVCFast(LinearClassifier):
      """
      Pegasos Support Vector Classifier with performance improvements
      """

      def __init__(self, n_iter=20, lam=1):
            """
            The constructor can optionally take a parameter n_iter specifying how
            many times we want to iterate through the training set.
            """
            self.n_iter = n_iter
            self.lam = lam

      def fit(self, X, Y):
            """
            Train a linear classifier using the perceptron learning algorithm.
            """

            self.find_classes(Y)

            Ye = self.encode_outputs(Y)

            if not isinstance(X, np.ndarray):
                  X = X.toarray()

            # Initialize the weight vector to all zeros.
            n_features = X.shape[1]
            m = X.shape[0]
            self.w = np.zeros(n_features)

            for i in range(1, self.n_iter + 1):

                  j = np.random.choice(m, 1)[0]
                  x, y = X[j], Ye[j]
                  nu = 1. / (self.lam * i)
                  #score = self.w.dot(x)
                  score = ddot(self.w, x)
                  if y * score < 1:
                        #self.w = (1 - nu * self.lam) * self.w + (nu * y) * x
                        #self.w = dscal((1 - nu * self.lam), self.w) + dscal(nu * y, x)
                        dscal((1 - nu * self.lam), self.w)
                        #y += a*x == y + a*x
                        daxpy(y=self.w, x=x, a=nu*y)
                  else:
                        #x *= 3 == x = x * 3
                        #self.w *= (1 - nu * self.lam)
                        #self.w = (1 - nu * self.lam) * self.w
                        #self.w = dscal((1 - nu * self.lam), self.w)
                        dscal((1 - nu * self.lam), self.w)

In [47]:
pipeline = make_pipeline(
      TfidfVectorizer(),
      SelectKBest(k=1000),
      Normalizer(),
      PegasosSVCFast(lam=1/len(Xtrain), n_iter=100000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 2.04 sec.
Accuracy: 0.8389.


In [48]:
pipeline = make_pipeline(
      TfidfVectorizer(),
      SelectKBest(k=1000),
      Normalizer(),
      PegasosSVC(lam=1/len(Xtrain), n_iter=100000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 2.06 sec.
Accuracy: 0.8321.


In [49]:
def add_sparse_to_dense(x, w, factor):
      """
      Adds a sparse vector x, scaled by some factor, to a dense vector.
      This can be seen as the equivalent of w += factor * x when x is a dense
      vector.
      """
      w[x.indices] += factor * x.data


def sparse_dense_dot(x, w):
      """
      Computes the dot product between a sparse vector x and a dense vector w.
      """
      return np.dot(w[x.indices], x.data)

class PegasosSVCSparse(LinearClassifier):
      """
      Pegasos Support Vector classifier using sparse matrices
      """

      def __init__(self, n_iter=20, lam=1):
            """
            The constructor can optionally take a parameter n_iter specifying how
            many times we want to iterate through the training set.
            """
            self.n_iter = n_iter
            self.lam = lam

      def fit(self, X, Y):
            """
            Train a linear classifier using the perceptron learning algorithm.
            """

            self.find_classes(Y)

            Ye = self.encode_outputs(Y)


            # Initialize the weight vector to all zeros.
            n_features = X.shape[1]
            m = X.shape[0]
            self.w = np.zeros(n_features)
            for i in range(1, self.n_iter + 1):

                  j = np.random.choice(m, 1)[0]
                  x, y = X[j], Ye[j]
                  nu = 1. / (self.lam * i)
                  score = sparse_dense_dot(x=x, w=self.w)
                  self.w = (1 - nu * self.lam) * self.w
                  if y * score < 1:
                        add_sparse_to_dense(x = x, w = self.w, factor = (nu * y))


In [50]:
pipeline = make_pipeline(
      TfidfVectorizer(),
      Normalizer(),
      PegasosSVCSparse(lam=1/len(Xtrain), n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.38 sec.
Accuracy: 0.8141.


In [51]:
class PegasosSVCSparseFastScaling(LinearClassifier):
      """
      Pegasos Support Vector Classifier using sparse matrices and performance improvements
      """

      def __init__(self, n_iter=20, lam=1):
            """
            The constructor can optionally take a parameter n_iter specifying how
            many times we want to iterate through the training set.
            """
            self.n_iter = n_iter
            self.lam = lam

      def fit(self, X, Y):
            """
            Train a linear classifier using the perceptron learning algorithm.
            """

            self.find_classes(Y)

            Ye = self.encode_outputs(Y)


            # Initialize the weight vector to all zeros.
            n_features = X.shape[1]
            m = X.shape[0]
            self.w = np.zeros(n_features)


            a = 1
            for i in range(2, self.n_iter + 2):

                  j = np.random.choice(m, 1)[0]
                  x, y = X[j], Ye[j]
                  nu = 1. / (self.lam * i)
                  score = a * sparse_dense_dot(x=x, w=self.w)

                  if y * score < 1:
                        add_sparse_to_dense(x = x, w = self.w, factor = (nu * y) /a)
                  a = (1 - nu * self.lam) * a
            self.w = a * self.w

In [52]:
pipeline = make_pipeline(
      TfidfVectorizer(),
      Normalizer(),
      PegasosSVCSparseFastScaling(lam=1/len(Xtrain), n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.18 sec.
Accuracy: 0.8120.
